In [ ]:
global Path  
Path="file:/home/spark/ntcu_workshop/"
log_file_path=Path+"data/NASA_access_log_Aug95"
base_df = spark.read.text(log_file_path)

In [ ]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.count()

In [ ]:
logs_df = split_df.filter(~split_df['host'].isNull() &
                              ~split_df['timestamp'].isNull() &
                              ~split_df['path'].isNull() &
                              ~split_df['status'].isNull() &
                             ~split_df['content_size'].isNull())
logs_df.printSchema()
logs_rdd = logs_df.rdd
logs_rdd.count()

In [ ]:
not_found_rdd = logs_rdd.filter(lambda r: r['status'] == 304)
not_found_rdd.cache()

print('Found {0} 304 URLs').format(not_found_rdd.count())

In [ ]:
top_20_not_found_rdd = not_found_rdd.map(lambda r: (r['path'],1))\
                                    .reduceByKey(lambda x,y: x+y)\
                                    .sortBy(lambda kv: kv[1], ascending=False)
top_20_not_found_rdd.take(20)

In [ ]:
hosts_304_count_rdd = not_found_rdd.map(lambda r: (r['host'],1))\
                                   .reduceByKey(lambda x,y: x+y)\
                                   .takeOrdered(20, key = lambda x: -x[1])
        
print 'Top 25 hosts that generated errors:\n'
hosts_304_count_rdd